# Homework 6: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [10]:
library(tidyverse)

setwd("C:/Users/MCT40/OneDrive - University of Pittsburgh/Classes/Spring2022/CMU_STATS/HOMEWORK")
LD <- read.csv("lexDat/LexicalData.csv")
items <- read.csv("lexDat/items.csv")

LD_nona <- LD
LD_nona$D_RT <- as.numeric(gsub(",", "", LD$D_RT)) #remove the commas and make the col numeric
LD_nona %>% filter(D_RT !="n/a") -> LD2 #remove NAs save to LD2

any(LD2$D_RT == "n/a")

LD2$Word <- LD$D_Word #add a new column identical to D_Word just called word to join the two databases
#I added a new column instead of renaming it because I wasn't sure this was correct
# In other iterationa I would simply rename this column using the "names" function

LD3 <- left_join(LD2, items, by='Word') #adding together all of the variables to make the final question of the homework easier
head(LD3)

[1] FALSE

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore,Word,Occurrences,Length,Freq_HAL,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437,browse,2,6,"7,016",8.856
2,67,1,1,1094,refrigerant,false,0.825,refrigerant,3,11,104,4.644
3,120,1,1,587,gaining,false,-0.645,gaining,4,7,"4,039",8.304
4,21,1,1,984,cheerless,false,0.025,cheerless,4,9,14,2.639
5,236,1,1,577,pattered,false,-0.763,pattered,4,8,4,1.386
6,236,2,1,715,conjures,false,-0.364,conjures,4,8,194,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [11]:
ML1 <- lm(D_RT ~ Log_Freq_HAL * Length, data= LD3)
summary(ML1)
AIC(ML1)


Call:
lm(formula = D_RT ~ Log_Freq_HAL * Length, data = LD3)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


[1] 914436.4

Now, install `lme4` using `install.packages()` and then load the library. 

In [12]:
#install.packages("lme4") because I ran this a few times
library(lme4)



Warning message:
"package 'lme4' is in use and will not be installed"


Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [13]:
ML2 <- lmer(D_RT ~ Log_Freq_HAL * Length + (1|Sub_ID), data= LD3)
summary(ML2)
AIC(ML2)



Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: LD3

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

[1] 888247.6

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> The t-values are identical in both models (really tiny: < 2e-16) except for the Log_Freq_Hal value which drops from 0.00221 to 2.63e-06 from the linear model to the fixed effect. This indicates that we have a more conservative model in the mixed-effects case. 
This might be due to accounting for the variability provided by the random effects. 

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [16]:
AIC(ML1) # Linear model
AIC(ML2) # MEM model

[1] 914436.4

[1] 888247.6

> The second model (using lme4::lmer) is a better fit because the AIC values are lower.
 

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> Word and number of occurences would be interesting factors to use as random effects.

**DUE:** 5pm EST, March 25, 2022

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> Dr. (to be) Emily Goldberg, because... she knows what it is